In [1]:
import pylab as pl
import pandas as pd
import geopandas as gp
import numpy as np
import scipy.stats
import os
import json
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
#lowercase ols
import statsmodels.formula.api as smf
#uppercase OLS
import datetime as dt
import re


%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 1. Define Research Question

### Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
 - Acquiured data from CUSP data facility for 311 complaints from 2010 to present

In [2]:
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11224,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,NaN,NaN,NaN,1-1-1173130914,NaN,NaN,NaN,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,Street,11234,1157 EAST 57 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11218,722 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#data.columns

In [5]:
comdistcalls = data[['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor','Incident Zip',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'Borough','Latitude', 'Longitude', 'Location']]

In [6]:
comdistcalls.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,Borough,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11224,BROOKLYN,NaN,NaN,Open,NaN,NaN,11/05/2015 02:59:15 AM,13 BROOKLYN,BROOKLYN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,NaN,Email Sent,11/19/2015 02:10:47 AM,Your comments have been submitted to the Mayor...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,11234,BROOKLYN,NaN,NaN,Open,12/05/2015 02:06:51 AM,The Department of Parks and Recreation usually...,NaN,18 BROOKLYN,BROOKLYN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,11218,BROOKLYN,NaN,Precinct,Assigned,11/05/2015 10:02:20 AM,Your complaint has been received by the Police...,11/05/2015 02:07:18 AM,12 BROOKLYN,BROOKLYN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NaN,NaN,NaN,NaN,Closed,NaN,The Human Resources Administration received yo...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN


In [7]:
#checking for all Boroughs in data

comdistcalls['Borough'].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [8]:
# Checking for all Community Board Districts

comdistcalls['Community Board'].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

In [9]:
#length of unniqe values
len(comdistcalls['Community Board'].unique())

# There are 77 because of the "Unspecified" borough districts and non-real community board. 

77

In [10]:
#Check for dupicates, must be by unique identifier

comdistcalls['dupcheck'] = comdistcalls.duplicated('Unique Key')

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
comdistcalls.groupby(['dupcheck']).count()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,Borough,Latitude,Longitude,Location
dupcheck,,,,,,,,,,,,,,,,,,,,
False,10187744,10187744,9736865,10187744,10187744,10187744,10156687,9393585,9399032,7310,1608281,10187744,3067458,6401005,9936812,10187744,10187744,9308992,9308992,9308992
True,22,22,22,22,22,22,22,22,22,0,0,22,22,10,20,22,22,22,22,22


In [12]:
#there are only 22 duplicates,  drop
comdistcalls.drop_duplicates('Unique Key')

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,City,Landmark,...,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,Borough,Latitude,Longitude,Location,dupcheck
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11224,BROOKLYN,NaN,...,Open,NaN,NaN,11/05/2015 02:59:15 AM,13 BROOKLYN,BROOKLYN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)",False
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,Email Sent,11/19/2015 02:10:47 AM,Your comments have been submitted to the Mayor...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN,False
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,11234,BROOKLYN,NaN,...,Open,12/05/2015 02:06:51 AM,The Department of Parks and Recreation usually...,NaN,18 BROOKLYN,BROOKLYN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)",False
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,11218,BROOKLYN,NaN,...,Assigned,11/05/2015 10:02:20 AM,Your complaint has been received by the Police...,11/05/2015 02:07:18 AM,12 BROOKLYN,BROOKLYN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)",False
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NaN,NaN,NaN,...,Closed,NaN,The Human Resources Administration received yo...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN,False
5,31914148,11/05/2015 01:55:55 AM,NaN,DOT,Department of Transportation,Highway Condition,Pothole - Highway,10304,STATEN ISLAND,NaN,...,Open,11/15/2015 01:55:55 AM,The Department of Transportation requires 10 d...,NaN,02 STATEN ISLAND,STATEN ISLAND,40.606875,-74.085408,"(40.60687536641399, -74.0854077221027)",False
6,31912764,11/05/2015 01:55:37 AM,NaN,DPR,DPR,Agency Issues,New Tree Complaint,NaN,NaN,NaN,...,Email Sent,11/19/2015 01:59:29 AM,Your comments have been submitted to the Depar...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN,False
7,31912611,11/05/2015 01:47:12 AM,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,10021,NEW YORK,NaN,...,Open,11/05/2015 09:47:12 AM,Your complaint has been forwarded to the New Y...,NaN,08 MANHATTAN,MANHATTAN,40.769379,-73.957930,"(40.769378795761845, -73.95793023233308)",False
8,31913909,11/05/2015 01:39:31 AM,11/05/2015 02:35:44 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,11231,BROOKLYN,NaN,...,Closed,11/05/2015 09:39:31 AM,The Police Department responded to the complai...,11/05/2015 02:33:52 AM,06 BROOKLYN,BROOKLYN,40.682614,-73.993240,"(40.682613541568664, -73.99323966551619)",False
9,31914856,11/05/2015 01:38:35 AM,11/05/2015 02:33:33 AM,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,11231,BROOKLYN,NaN,...,Closed,11/05/2015 09:38:35 AM,The Police Department responded to the complai...,11/05/2015 02:28:24 AM,06 BROOKLYN,BROOKLYN,40.682614,-73.993240,"(40.682613541568664, -73.99323966551619)",False


In [13]:
len(comdistcalls)

10187766

In [14]:
duplicates = data[data.duplicated('Unique Key')]

In [15]:
#Checking for total NaN values

comdistcalls.isnull().sum()

Unique Key                               0
Created Date                             0
Closed Date                         450879
Agency                                   0
Agency Name                              0
Complaint Type                           0
Descriptor                           31057
Incident Zip                        794159
City                                788712
Landmark                          10180456
Facility Type                      8579485
Status                                   0
Due Date                           7120286
Resolution Description             3786751
Resolution Action Updated Date      250934
Community Board                          0
Borough                                  0
Latitude                            878752
Longitude                           878752
Location                            878752
dupcheck                                 0
dtype: int64

### Patterns in missing values
 - Many of these columns such as "Landmark" and "Facility Type" have many NaN values.  
 - Other fields such as "Unique Key" which is probably an auto-generated field has zero
 - This pattern may be due to REQUIRED and OPTIONAL fields when submitting a complaint

### Data Editing:
 - Replacing missing "Resolution Descriptor" with "Complaint Type" is possible, as Complaint Type is probably the most similar value. However, leaving the field blank in this case may not affect the analysis. 
 - below I will go ahead and change these values

In [16]:
comdistcalls["Resolution Description"].fillna(comdistcalls["Complaint Type"], inplace=True)


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
comdistcalls.isnull().sum()

Unique Key                               0
Created Date                             0
Closed Date                         450879
Agency                                   0
Agency Name                              0
Complaint Type                           0
Descriptor                           31057
Incident Zip                        794159
City                                788712
Landmark                          10180456
Facility Type                      8579485
Status                                   0
Due Date                           7120286
Resolution Description                   0
Resolution Action Updated Date      250934
Community Board                          0
Borough                                  0
Latitude                            878752
Longitude                           878752
Location                            878752
dupcheck                                 0
dtype: int64

As shown in the above output, there are no longer NaNs in the Complaint Type columnm

In [18]:
#This function will mark the "Unspecified" CB's with 0
def markers(data):
    if re.search(r"Unspecified", data):
        return 0
    elif int(re.search(r"(\d\d*)\s\w+", data).group(1))>18:
        return 0
    else:
        return 1

In [19]:
comdistcalls2  = comdistcalls["Community Board"].apply(markers)
#applying the function in a new 1-D array

In [20]:
comdistcalls["comdistcalls2"] = comdistcalls2

#adding a column to the original DF with the vlaues from the function

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
comdistcalls.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Incident Zip,City,Landmark,...,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,Borough,Latitude,Longitude,Location,dupcheck,comdistcalls2
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,11224,BROOKLYN,NaN,...,NaN,Street Condition,11/05/2015 02:59:15 AM,13 BROOKLYN,BROOKLYN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)",False,1
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,11/19/2015 02:10:47 AM,Your comments have been submitted to the Mayor...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN,False,0
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,11234,BROOKLYN,NaN,...,12/05/2015 02:06:51 AM,The Department of Parks and Recreation usually...,NaN,18 BROOKLYN,BROOKLYN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)",False,1
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,11218,BROOKLYN,NaN,...,11/05/2015 10:02:20 AM,Your complaint has been received by the Police...,11/05/2015 02:07:18 AM,12 BROOKLYN,BROOKLYN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)",False,1
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NaN,NaN,NaN,...,NaN,The Human Resources Administration received yo...,NaN,0 Unspecified,Unspecified,NaN,NaN,NaN,False,0


In [22]:
comdistcallsREDUX = comdistcalls[comdistcalls.comdistcalls2 !=0]

In [23]:
#comdistcallsREDUX.head()

In [24]:
len(comdistcallsREDUX['Community Board'].unique())

59

In [25]:
pivot = comdistcallsREDUX["Unique Key"].groupby(comdistcallsREDUX["Community Board"]).count()

In [26]:
pivot.head(20)

Community Board
01 BRONX             74631
01 BROOKLYN         185057
01 MANHATTAN         77974
01 QUEENS           171484
01 STATEN ISLAND    182713
02 BRONX             60257
02 BROOKLYN         121022
02 MANHATTAN        133860
02 QUEENS           114333
02 STATEN ISLAND    121132
03 BRONX             75134
03 BROOKLYN         197306
03 MANHATTAN        150296
03 QUEENS           122009
03 STATEN ISLAND    136487
04 BRONX            181953
04 BROOKLYN         139706
04 MANHATTAN        132660
04 QUEENS            99316
05 BRONX            161078
Name: Unique Key, dtype: int64

In [27]:
# must create a data frame from the pivot table above

CBRank = pd.DataFrame(pivot)

In [28]:
CBRank.head()

,Unique Key
Community Board,
01 BRONX,74631
01 BROOKLYN,185057
01 MANHATTAN,77974
01 QUEENS,171484
01 STATEN ISLAND,182713


In [29]:
CBRank["RANK"] = CBRank["Unique Key"].rank(ascending = False)

In [30]:
CBRank.head()

,Unique Key,RANK
Community Board,,
01 BRONX,74631,57.0
01 BROOKLYN,185057,9.0
01 MANHATTAN,77974,55.0
01 QUEENS,171484,12.0
01 STATEN ISLAND,182713,10.0


In [31]:
CBRank = CBRank.reset_index()
#the column labels are off, must RESET index
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html

In [32]:
CBRank.head()

,Community Board,Unique Key,RANK
0,01 BRONX,74631,57.0
1,01 BROOKLYN,185057,9.0
2,01 MANHATTAN,77974,55.0
3,01 QUEENS,171484,12.0
4,01 STATEN ISLAND,182713,10.0


In [40]:
#Renaming Column
CBRank.rename(columns={'Unique Key': 'Total Complaints'}, inplace=True)
CBRank.head(59)

,Community Board,Total Complaints,RANK
0,01 BRONX,74631,57.0
1,01 BROOKLYN,185057,9.0
2,01 MANHATTAN,77974,55.0
3,01 QUEENS,171484,12.0
4,01 STATEN ISLAND,182713,10.0
5,02 BRONX,60257,59.0
6,02 BROOKLYN,121022,40.0
7,02 MANHATTAN,133860,32.0
8,02 QUEENS,114333,41.0
9,02 STATEN ISLAND,121132,39.0


### As shown in the RANK Table above, Community Board 12 from Manhattan is ranked #1 with 268,053 complaints

# 4. Interpret Results